## 3.6　手書き数字認識

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd

### 3.6.1　MNIST データセット

以下から「deep-learning-from-scratch」をダウンロードする。

・https://github.com/oreilly-japan/deep-learning-from-scratch

現在のディレクトリ（current working directory）を確認する。

In [2]:
os.getcwd()

'/Users/yuta.shimizu/Downloads/ML'

書籍通り、現在のディレクトリを ch01 に変更する。

In [3]:
os.chdir('/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master/ch01')
os.getcwd()

'/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master/ch01'

os.pardir は、親ディレクトリ（deep-learning-from-scratch-master）の代わりとして使用できる。

In [4]:
os.pardir

'..'

モジュール検索パスを追加する。

In [5]:
sys.path.append(os.pardir)

親ディレクトリ（deep-learning-from-scratch-master）の中身を、モジュールとしてインポートする。

In [6]:
from dataset.mnist import load_mnist

In [7]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [8]:
from PIL import Image

In [9]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [10]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [11]:
img = x_train[0]
label = t_train[0]
print(label)

5


In [12]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

(784,)
(28, 28)


In [13]:
img_show(img)

### 3.6.2　ニューラルネットワークの推論処理

In [14]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

normalize = True で、データの値が 0.0~1.0 の範囲に収まるように変換される。（正規化）

In [15]:
def init_network():
    with open('/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master/ch03/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

In [16]:
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [18]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [19]:
x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


np.argmax は、配列中の最大値のインデックスを取得する。

### 3.6.3　バッチ処理

In [20]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


画像の元データは $ (28 \times 28) $ だが、784 の要素からなる 1 次元配列 $(1 \times 784) $ に変換して入力されている。<br>
これは 1 枚あたりのデータサイズなので、100 枚になれば $ (100 \times 784) $ になる。

コンピュータで計算する上で、バッチ処理は効率良く処理し、負担を軽減することができる。

In [21]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [22]:
print(list(range(0, 10)))
print(list(range(0, 10, 3)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 3, 6, 9]


最大値のインデックスを取得しているのが分かる。

In [23]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


In [25]:
y = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])

print(y==t)
print(np.sum(y==t))

[ True  True False  True]
3


### 参考文献

・[MNIST データセットの読み込み](https://www.anarchive-beta.com/entry/2020/06/09/180000)<br>
・[Kaggle でのデータインポート](https://www.kaggle.com/yshiml/zero-3-06)<br>
・[Pythonでimportの対象ディレクトリのパスを確認・追加（sys.pathなど）](https://note.nkmk.me/python-import-module-search-path/)